# SVM

In [17]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error

#경고 무시
import warnings 
warnings.filterwarnings('ignore')

matplotlib.rc('font',family='Malgun Gothic')
matplotlib.rc('axes', unicode_minus = False)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# 예측모델

## 8:2 split - pac투입률

In [18]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PAC 투입률','Cluster']]
df_raw = df_raw[df_raw['Cluster']==2]
df_raw = df_raw.drop('Cluster',axis=1)
df_raw

,탁도,pH,수온,전기전도도,알칼리도,원수유입유량,PAC 투입률
2074,21.4113,7.6402,12.2816,307.2862,29.9387,3921.2190,31.590355
2075,22.0186,7.6209,12.3007,305.8224,30.7273,2653.9253,31.174351
2076,21.3893,7.6359,12.3980,304.0589,30.6737,3962.9587,31.650393
2077,23.1375,7.6359,12.3274,301.9487,29.5469,3954.5662,31.617223
2078,23.7357,7.6441,12.8119,300.7706,28.4299,3934.3481,31.903811
...,...,...,...,...,...,...,...
82975,13.0521,7.4637,25.7801,238.3427,33.8488,3654.5208,35.004589
82976,14.3865,7.4841,25.8252,240.2771,33.9713,3658.0729,34.943262
83720,163.3635,7.1861,23.4300,133.5990,33.5707,3719.8125,65.413243
83721,153.5927,7.0712,23.4455,135.7146,33.8107,3466.3542,58.425939


In [19]:
#설명,목표 변수 데이터 구분
df_raw_x = df_raw.drop("PAC 투입률", axis = 1, inplace = False)
scaler = StandardScaler()
df_raw_x = scaler.fit_transform(df_raw_x)

df_raw_y = df_raw["PAC 투입률"]
#데이터 분할
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_raw_x,df_raw_y,test_size=0.2,random_state=6666)
print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (51855, 6)
분할 후 설명변수 데이터:Train (41484, 6) Test (10371, 6)


In [20]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 6.056
RMSE on test set: 6.009
MAE on training set: 4.064
MAE on test set: 4.048
Score on training set: 0.662
Score on test set: 0.660


## 8:2 split - pac투입량

In [21]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PAC 투입량','Cluster']]
df_raw = df_raw[df_raw['Cluster']==2]
df_raw = df_raw.drop('Cluster',axis=1)
df_raw

,탁도,pH,수온,전기전도도,알칼리도,원수유입유량,PAC 투입량
2074,21.4113,7.6402,12.2816,307.2862,29.9387,3921.2190,123.8727
2075,22.0186,7.6209,12.3007,305.8224,30.7273,2653.9253,82.7344
2076,21.3893,7.6359,12.3980,304.0589,30.6737,3962.9587,125.4292
2077,23.1375,7.6359,12.3274,301.9487,29.5469,3954.5662,125.0324
2078,23.7357,7.6441,12.8119,300.7706,28.4299,3934.3481,125.5207
...,...,...,...,...,...,...,...
82975,13.0521,7.4637,25.7801,238.3427,33.8488,3654.5208,127.9250
82976,14.3865,7.4841,25.8252,240.2771,33.9713,3658.0729,127.8250
83720,163.3635,7.1861,23.4300,133.5990,33.5707,3719.8125,243.3250
83721,153.5927,7.0712,23.4455,135.7146,33.8107,3466.3542,202.5250


In [22]:
#설명,목표 변수 데이터 구분
df_raw_x = df_raw.drop("PAC 투입량", axis = 1, inplace = False)
scaler = StandardScaler()
df_raw_x = scaler.fit_transform(df_raw_x)
df_raw_y = df_raw["PAC 투입량"]
#데이터 분할
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_raw_x,df_raw_y,test_size=0.2,random_state=6666)
print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (51855, 6)
분할 후 설명변수 데이터:Train (41484, 6) Test (10371, 6)


In [23]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 18.714
RMSE on test set: 18.962
MAE on training set: 12.424
MAE on test set: 12.544
Score on training set: 0.751
Score on test set: 0.745


## 8:2 split - pacs투입률

In [24]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PACS 투입률','Cluster']]
df_raw = df_raw[df_raw['Cluster']==1]
df_raw = df_raw.drop('Cluster',axis=1)
df_raw

,탁도,pH,수온,전기전도도,알칼리도,원수유입유량,PACS 투입률
0,7.7017,7.6528,2.5044,314.8864,99.5002,2425.8030,19.440161
1,7.2569,7.6505,2.4357,316.8224,99.5422,2423.7048,19.246073
2,6.9190,7.6477,2.6493,318.1849,99.5546,2441.6343,19.098519
3,6.9069,7.6480,2.3842,319.0541,99.5257,2442.0156,19.233006
4,6.7461,7.6477,2.4758,320.2157,99.5107,2435.9121,19.553661
...,...,...,...,...,...,...,...
81457,5.8844,7.5648,16.9494,518.6604,66.1711,2455.0313,20.692201
81532,4.9385,7.5351,17.4355,454.0615,64.3489,3239.7917,22.254517
81597,5.3990,7.5163,17.0766,437.6427,64.4198,2465.1354,20.729084
81631,5.5719,7.5185,16.6657,437.6708,64.3520,3535.3958,20.789751


In [25]:
#설명,목표 변수 데이터 구분
df_raw_x = df_raw.drop("PACS 투입률", axis = 1, inplace = False)
scaler = StandardScaler()
df_raw_x = scaler.fit_transform(df_raw_x)
df_raw_y = df_raw["PACS 투입률"]
#데이터 분할
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_raw_x,df_raw_y,test_size=0.2,random_state=6666)
print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (4332, 6) Test (1084, 6)


In [26]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 3.116
RMSE on test set: 3.016
MAE on training set: 1.787
MAE on test set: 1.676
Score on training set: 0.789
Score on test set: 0.800


## 8:2 split - pacs투입량

In [27]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PACS 투입량','Cluster']]
df_raw = df_raw[df_raw['Cluster']==1]
df_raw = df_raw.drop('Cluster',axis=1)
df_raw

,탁도,pH,수온,전기전도도,알칼리도,원수유입유량,PACS 투입량
0,7.7017,7.6528,2.5044,314.8864,99.5002,2425.8030,47.1580
1,7.2569,7.6505,2.4357,316.8224,99.5422,2423.7048,46.6468
2,6.9190,7.6477,2.6493,318.1849,99.5546,2441.6343,46.6316
3,6.9069,7.6480,2.3842,319.0541,99.5257,2442.0156,46.9673
4,6.7461,7.6477,2.4758,320.2157,99.5107,2435.9121,47.6310
...,...,...,...,...,...,...,...
81457,5.8844,7.5648,16.9494,518.6604,66.1711,2455.0313,50.8000
81532,4.9385,7.5351,17.4355,454.0615,64.3489,3239.7917,72.1000
81597,5.3990,7.5163,17.0766,437.6427,64.4198,2465.1354,51.1000
81631,5.5719,7.5185,16.6657,437.6708,64.3520,3535.3958,73.5000


In [28]:
#설명,목표 변수 데이터 구분
df_raw_x = df_raw.drop("PACS 투입량", axis = 1, inplace = False)
scaler = StandardScaler()
df_raw_x = scaler.fit_transform(df_raw_x)
df_raw_y = df_raw["PACS 투입량"]
#데이터 분할
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_raw_x,df_raw_y,test_size=0.2,random_state=6666)
print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (4332, 6) Test (1084, 6)


In [29]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 12.392
RMSE on test set: 11.883
MAE on training set: 6.587
MAE on test set: 6.409
Score on training set: 0.806
Score on test set: 0.829


## 시계열 - pac투입률

In [30]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PAC 투입률','Cluster']]

df_train = df_raw.iloc[:76186]
df_test = df_raw.iloc[76186:]
df_train = df_train[df_train['Cluster']==2]
df_test = df_test[df_test['Cluster']==2]
df_train_x = df_train.drop("PAC 투입률", axis = 1, inplace = False)
df_train_x = scaler.fit_transform(df_train_x)
df_train_y = df_train["PAC 투입률"]
df_test_x = df_test.drop("PAC 투입률", axis = 1, inplace = False)
df_test_x = scaler.fit_transform(df_test_x)
df_test_y = df_test["PAC 투입률"]


In [31]:
print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (49222, 7) Test (2633, 7)


In [32]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 5.970
RMSE on test set: 6.597
MAE on training set: 4.026
MAE on test set: 4.620
Score on training set: 0.675
Score on test set: -0.061


## 시계열 - pac투입량

In [33]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PAC 투입량','Cluster']]
df_train = df_raw.iloc[:76186]
df_test = df_raw.iloc[76186:]
df_train = df_train[df_train['Cluster']==2]
df_test = df_test[df_test['Cluster']==2]
df_train_x = df_train.drop("PAC 투입량", axis = 1, inplace = False)
df_train_x = scaler.fit_transform(df_train_x)
df_train_y = df_train["PAC 투입량"]
df_test_x = df_test.drop("PAC 투입량", axis = 1, inplace = False)
df_test_x = scaler.fit_transform(df_test_x)
df_test_y = df_test["PAC 투입량"]

print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (49222, 7) Test (2633, 7)


In [34]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 18.603
RMSE on test set: 17.096
MAE on training set: 12.350
MAE on test set: 12.140
Score on training set: 0.759
Score on test set: 0.489


## 8:2 split - pacs투입률

In [35]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PACS 투입률','Cluster']]
df_train = df_raw.iloc[:76186]
df_test = df_raw.iloc[76186:]
df_train = df_train[df_train['Cluster']==1]
df_test = df_test[df_test['Cluster']==1]
df_train_x = df_train.drop("PACS 투입률", axis = 1, inplace = False)
df_train_x = scaler.fit_transform(df_train_x)
df_train_y = df_train["PACS 투입률"]
df_test_x = df_test.drop("PACS 투입률", axis = 1, inplace = False)
df_test_x = scaler.fit_transform(df_test_x)
df_test_y = df_test["PACS 투입률"]

print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (5337, 7) Test (79, 7)


In [36]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 2.979
RMSE on test set: 7.060
MAE on training set: 1.692
MAE on test set: 5.496
Score on training set: 0.807
Score on test set: -6.204


## 8:2 시계열 - pacs투입량

In [37]:
df_raw = pd.read_excel("data_cluster.xlsx")
df_raw = df_raw[['탁도','pH','수온','전기전도도','알칼리도','원수유입유량','PACS 투입량','Cluster']]
df_train = df_raw.iloc[:76186]
df_test = df_raw.iloc[76186:]
df_train = df_train[df_train['Cluster']==1]
df_test = df_test[df_test['Cluster']==1]
df_train_x = df_train.drop("PACS 투입량", axis = 1, inplace = False)
df_train_x = scaler.fit_transform(df_train_x)
df_train_y = df_train["PACS 투입량"]
df_test_x = df_test.drop("PACS 투입량", axis = 1, inplace = False)
df_test_x = scaler.fit_transform(df_test_x)
df_test_y = df_test["PACS 투입량"]

print("분할 전 설명변수 데이터:",df_raw_x.shape)
print("분할 후 설명변수 데이터:Train",df_train_x.shape,"Test",df_test_x.shape)

분할 전 설명변수 데이터: (5416, 6)
분할 후 설명변수 데이터:Train (5337, 7) Test (79, 7)


In [38]:
# 모델 초기화 및 학습
svm_uncustomized = SVR()
svm_uncustomized.fit(df_train_x, df_train_y)

# 예측값 계산
y_pred_train = svm_uncustomized.predict(df_train_x)
y_pred_test = svm_uncustomized.predict(df_test_x)

# RMSE 계산
train_rmse = np.sqrt(mean_squared_error(df_train_y, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(df_test_y, y_pred_test))

# MAE 계산
train_mae = mean_absolute_error(df_train_y, y_pred_train)
test_mae = mean_absolute_error(df_test_y, y_pred_test)

# 성능 출력
print("RMSE on training set: {:.3f}".format(train_rmse))
print("RMSE on test set: {:.3f}".format(test_rmse))
print("MAE on training set: {:.3f}".format(train_mae))
print("MAE on test set: {:.3f}".format(test_mae))
print("Score on training set: {:.3f}".format(svm_uncustomized.score(df_train_x, df_train_y)))
print("Score on test set: {:.3f}".format(svm_uncustomized.score(df_test_x, df_test_y)))

RMSE on training set: 11.879
RMSE on test set: 14.348
MAE on training set: 6.250
MAE on test set: 9.810
Score on training set: 0.825
Score on test set: -0.216
